# High performance containers – the ```collections``` module

The collections module is the high performace alternative for built-in data structures (lists, tuples, sets and dicts):

- deque: 
    - Supports rotation and reverse operations.
- defaultdict:
    - Uses type factories to provide default values to dictionary keys.
- ordereddict:
    - A mix between lists and dicts.
- counter:
    - Gives us a counter for elements in an iterable.
- chainmap:
    - Groups many dicts in only one object.

## Deques

In [1]:
from collections import deque
import random, timeit

### Rotation

#### Rotation in lists

In [2]:
def list_rotate(seq1, n):
    """ Rotate a list left by n """
    # E.g: rotate([1,2,3,4,5], 2) => [4,5,1,2,3]
    return seq1[-n:] + seq1[:-n]

In [41]:
seq = random.sample(range(0, 10000), 1000)

In [4]:
timeit.timeit("list_rotate(seq, 10)", setup="from __main__ import list_rotate, seq")

6.5383076460002485

#### Rotation in deques

In [43]:
deque_seq = deque(seq)

In [6]:
timeit.timeit("deque_seq.rotate(10)", setup="from __main__ import deque_seq")

0.12837861300067743

Performance for rotation is better in deques than lists.

### Insert

#### Insert in lists

In [44]:
seq_1 = random.sample(range(0, 10000), 1000)

In [8]:
timeit.timeit("seq_1.insert(500,9999)", setup="from __main__ import seq_1")

297.45836762600084

#### Insert in deques

In [45]:
deque_seq_1 = deque(seq_1)

In [10]:
timeit.timeit("deque_seq_1.insert(500,9999)", setup="from __main__ import deque_seq_1")

0.8769112850004603

## Defaultdict

In [27]:
from collections import defaultdict
s = [('orange', 3), ('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
d = defaultdict(list)

for k, v in s:
    d[k].append(v)
d

defaultdict(list,
            {'orange': [3], 'yellow': [1, 3], 'blue': [2, 4], 'red': [1]})

In [36]:
s = [('orange', 3), ('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
# pass data structure expected as the value schema
d = defaultdict(set)

for k, v in s:
    d[k].add(v)
d

defaultdict(set, {'orange': {3}, 'yellow': {1, 3}, 'blue': {2, 4}, 'red': {1}})

As we can see, defaultdict maintains the order of insertion in dictionary. Besides, if a key exists, adds the new value to the existing list.

## Ordereddict

Its name gives us a hint about its functionality. Maintains insertion order.

In [62]:
from collections import OrderedDict

# hint: defining a tuple as a key in a dictionary.
d = OrderedDict.fromkeys([('Colombia', 'Bogotá'), 'Peru', 'Ecuador'])
d

OrderedDict([(('Colombia', 'Bogotá'), None),
             ('Peru', None),
             ('Ecuador', None)])

Ordered dict can be understood as mix of dict and list. Supports methods:
- ``` popitem(last=True)``` default last value is True, in this case, performs as LIFO structure. On the other hand, if last is False performs as FIFO.
- ``` move_to_end(key, last=True) ``` receives *key* to move to the last index of the dict. If last is False, moves the value to the first one.

In [63]:
print(d.popitem(last=0), d.popitem())
d

(('Colombia', 'Bogotá'), None) ('Ecuador', None)


OrderedDict([('Peru', None)])

In [64]:
# Now, add a new item to dictionary.
d['Argentina'] = 'Paraná'
print(d)
d.move_to_end('Peru')
d

OrderedDict([('Peru', None), ('Argentina', 'Paraná')])


OrderedDict([('Argentina', 'Paraná'), ('Peru', None)])

Besides things looked above, this structure drops duplicate values. Look:

In [66]:
list_cycling_teams = ['Deckeunik', 'Ineos', 'Movistar', 'Movistar']
cycling_dict = OrderedDict.fromkeys(list_cycling_teams)
cycling_dict

OrderedDict([('Deckeunik', None), ('Ineos', None), ('Movistar', None)])

## Counter

This structure allows us to get the quantity of items, per example in a text. We need to pass an iterable or mapping as param in constructor.

In [70]:
from collections import Counter


paragraph = "tres tristes tigres estan comiendo en tres tristes platos"

# instead of this
quantities = {}
for word in paragraph.split():
    if word in quantities:
        quantities[word] += 1
    else:
        quantities[word] = 1

print(quantities)

# you can do ...
quantities_counter = Counter(paragraph.split())
print(quantities_counter)


{'tres': 2, 'tristes': 2, 'tigres': 1, 'estan': 1, 'comiendo': 1, 'en': 1, 'platos': 1}
Counter({'tres': 2, 'tristes': 2, 'tigres': 1, 'estan': 1, 'comiendo': 1, 'en': 1, 'platos': 1})


In [76]:
counter_word = Counter("otorrinonaringologo")
print(counter_word)

# most_common method
print(counter_word.most_common(2))

Counter({'o': 6, 'r': 3, 'n': 3, 'i': 2, 'g': 2, 't': 1, 'a': 1, 'l': 1})
[('o', 6), ('r', 3)]


## ChainMap

Groups many dicts in only one object. Writes, updates, and deletions only operate on the first mapping

In [16]:
from collections import ChainMap

# creating chainmap passing dicts in constructor
from_constructor =  ChainMap({"x":2}, {"y":3})
for key, value in from_constructor.items():
    print(key, value)

y 3
x 2


In [17]:
# creating chainmap throuhgt maps attribute
from_maps_attr = ChainMap()
from_maps_attr.maps = [{"a":1},{"b":4}]
for key, value in from_maps_attr.items():
    print(key, value)

b 4
a 1


In [20]:
# We can use new_child method to pass a dict that we want it's going to be the initial dict
pattern = ChainMap({"x":2}, {"y":3})
print(pattern)
as_new_child = pattern.new_child({"z":1})
print(as_new_child)

ChainMap({'x': 2}, {'y': 3})
ChainMap({'z': 1}, {'x': 2}, {'y': 3})
